Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

In [ ]:
!pip install minsearch

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

C:\Users\alexe\miniconda3\Lib\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.sparse import csr_matrix, issparse


In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [8]:
question = 'Can I still join the course?'

In [9]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
search_results = search(question)

In [12]:
prompt = build_prompt(question, search_results)

In [15]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



In [16]:
answer = llm(prompt)

In [17]:
print(answer)

Yes, you can still join the course after the start date. Even if you don't register, you're eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything for the last minute.


In [18]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [20]:
rag("How do I patch KDE under FreeBSD?")

'I’m sorry, but there is no information available in the context regarding how to patch KDE under FreeBSD.'

## "Agentic" RAG

In [22]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [30]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [31]:
prompt = prompt_template.format(question=question, context=context)

In [34]:
answer_json = llm(prompt)

In [35]:
import json

In [36]:
answer = json.loads(answer_json)

In [38]:
answer['action']

'SEARCH'

In [40]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [42]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [43]:
answer_json = llm(prompt)

In [44]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course even after the start date. Even if you haven't registered, you're eligible to submit homework. Just keep in mind that there will be deadlines for the final projects, so it's a good idea to stay on top of the deadlines and not leave things until the last minute.",
"source": "CONTEXT"
}


## Agentic Search

In [64]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [48]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [49]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [50]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [52]:
answer_json = llm(prompt)

In [82]:
answer = json.loads(answer_json)

In [75]:
previous_actions.append(answer)

In [76]:
keywords = answer['keywords']

In [77]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [78]:
search_results = dedup(search_results)

In [79]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [80]:
answer_json = llm(prompt)

In [83]:
print(answer['answer'])

To do well in Module 1 focused on Docker and Terraform, consider the following strategies:

1. **Understand Core Concepts**: Take time to understand the fundamental concepts of Docker and Terraform. Focus on how containers work in Docker and the infrastructure as code principles in Terraform.

2. **Hands-On Practice**: Engage in hands-on projects. Try to build your own Docker containers and deploy them. Similarly, create Terraform scripts to automate infrastructure deployment.

3. **Utilize Documentation**: Use the official documentation for both Docker and Terraform as a primary resource. They provide excellent examples and use cases that can reinforce your learning.

4. **Engage with the Community**: Participate in forums or community discussions related to Docker and Terraform. You can learn a lot from others' experiences and solutions to common problems.

5. **Study Regularly**: Create a study schedule that allows you to consistently review material and practice coding, rather than

In [ ]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

In [85]:
answer

{'action': 'ANSWER',
 'answer': "To be successful in Module 1, which focuses on Docker and Terraform, consider the following strategies: \n\n1. **Hands-on Practice**: Engage in hands-on projects to solidify your understanding of Docker and Terraform. Setting up your own Docker containers and writing Terraform scripts will help reinforce the concepts. \n\n2. **Resources**: Use official documentation for both Docker and Terraform extensively. They're comprehensive and can provide guidance on best practices. \n\n3. **Community Support**: Participate in forums and community groups related to Docker and Terraform. Platforms like Stack Overflow or specific Slack channels can be helpful for problem-solving and learning from others' experiences. \n\n4. **Time Management**: Allocate time regularly each week to study and practice. Break down the module into manageable sections and create a study schedule. \n\n5. **Study Groups**: Collaborate with peers for group study sessions. Explaining concep

In [86]:
iteration

3

## Function calling ("tool use")

In [87]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [89]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [126]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [121]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"module 1 tips for success"}', call_id='call_FcpWXGZqHeLqMecDQwLCMPXq', name='search', type='function_call', id='fc_686401a3efbc8191b3646e3ad1218ac80e0676c3b7e4712d', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"how to excel in module 1"}', call_id='call_PNjiVZq3Fe66ODLrgup0SaRm', name='search', type='function_call', id='fc_686401a4e5448191a158f243fe1518b80e0676c3b7e4712d', status='completed')]

In [122]:
calls = response.output

In [123]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [124]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_686401ebe13c81918f6f3c84a1e7cec00e0676c3b7e4712d', content=[ResponseOutputText(annotations=[], text='To do well in Module 1, here are some tips based on common challenges and solutions:\n\n1. **Understand Your Environment**:\n   - Ensure that you have set up your development environment correctly. This includes installing Docker and Terraform as indicated in the module resources.\n\n2. **Common Installation Issues**:\n   - If you encounter errors such as `ModuleNotFoundError: No module named \'psycopg2\'`, try installing it via:\n     ```bash\n     pip install psycopg2-binary\n     ```\n   - If you\'re still encountering issues, consider updating pip or conda:\n     ```bash\n     pip install --upgrade pip\n     ```\n     or\n     ```bash\n     conda update -n base -c defaults conda\n     ```\n\n3. **PostgreSQL Connectivity**:\n   - When connecting to PostgreSQL using SQLAlchemy, ensure your connection string is formatted correctly, for example:\n     ```p

In [ ]:
for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.text) 

In [128]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [129]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 How do I do well in module 1?


function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 tips"}', call_id='call_IED9lyUZOsS6ToxvZHLNQoN6', name='search', type='function_call', id='fc_686403087180819ebf792db8930e3f830594799b59703b57', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 successful strategies"}', call_id='call_WAvKTixmfYyoBrF89lS6N46R', name='search', type='function_call', id='fc_68640309279c819ea24ded9b9639c7330594799b59703b57', status='completed')

To excel in Module 1, which focuses on Docker and Terraform, here are some key strategies and tips based on common challenges faced by students:

1. **Understand Docker Basics**: Start by familiarizing yourself with the Docker ecosystem, including images, containers, and Docker Compose. Having a solid grasp of these concepts will help you avoid common pitfalls.

2. **Environment Setup**: Ensure you have a correctly set up local environment. This includes installing Docker and any necessary dependencies. Fo

 Docker and Terraform


function_call: ResponseFunctionToolCall(arguments='{"query":"Docker tips for Module 1"}', call_id='call_zcxJlRqgjfMjavLOgVY1FSj5', name='search', type='function_call', id='fc_6864031f32a0819e850cbe6692b6ab160594799b59703b57', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"Terraform tips for Module 1"}', call_id='call_4n2UHkf4UtPpg3XAM0V5WFfI', name='search', type='function_call', id='fc_6864031f61ec819eba1a01625d5a64430594799b59703b57', status='completed')

To do well in Module 1 focusing on Docker and Terraform, here are some targeted tips for both technologies:

### Docker Tips:
1. **Basic Understanding**: Ensure you have foundational knowledge of Docker, such as how to create and manage containers, and how to use Docker Compose for multi-container applications.

2. **Correct Installation**: If you encounter issues like `ModuleNotFoundError: No module named 'psycopg2'`, make sure that you're installing the required Python modules in your Docker envi

 stop


## Multiple tools

In [130]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-01 17:50:28--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: 'chat_assistant.py'

     0K ...                                                   100%  434K=0.008s

2025-07-01 17:50:28 (434 KB/s) - 'chat_assistant.py' saved [3485/3485]



In [134]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [135]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [131]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}}]

In [136]:
tools.add_tool(add_entry, add_entry_description)

In [137]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [132]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [138]:
chat.run()

You: How do I do well in module 1?


You: add this to the FAQ database


You: stop


Chat ended.


In [139]:
index.docs[-1]

{'question': 'How do I do well in module 1?',
 'text': '1. Understand the Basics: Ensure you grasp foundational concepts like Docker and Terraform.\n2. Practice Regularly: Engage with practical assignments to reinforce concepts.\n3. Utilize Resources: Refer to course materials, documentation, and suggested readings.\n4. Ask Questions: Reach out if you encounter difficulties; engaging with peers or instructors can clarify your understanding.\n5. Review Feedback: Take time to review feedback after completing tasks to guide improvements.\n6. Stay Organized: Keep your work organized to easily trace back errors.',
 'section': 'user added',
 'course': 'data-engineering-zoomcamp'}

In [140]:
index